## Download
do not run - takes hours

In [ ]:
%%bash
redbiom search metadata 'where sample_type == "stool"' > stool_samples
redbiom search metadata 'where sample_type == "Stool"' >> stool_samples
export CTX=Deblur-illumina-16S-v4-150nt-10d7e0
redbiom fetch samples --from stool_samples --context $CTX --output stool_sv.biom

## Preprocess
do not run - takes overnight

In [3]:
%%bash
qiime tools import --input-path 99_otus.fasta --output-path 99_otus.qza --type FeatureData[Sequence]
qiime feature-classifier extract-reads --i-sequences 99_otus.qza --p-f-primer GTGYCAGCMGCCGCGGTAA --p-r-primer GGACTACNVGGGTWTCTAAT --o-reads 99_otus_v4.qza
qiime tools export 99_otus_v4.qza --output-dir .
mv dna-sequences.fasta 99_otus_v4.fasta
biom table-ids --observations -i stool_sv.biom | awk '{print ">"$1"blast_rocks\n"$1}' > stool_sv.fasta
makeblastdb -in 99_otus_v4.fasta -dbtype nucl -out 99_otus_v4.db
blastn -num_threads 4 -query stool_sv.fasta -outfmt "6 qacc sacc" -db 99_otus_v4.db -max_target_seqs 1 -out stool_sv_map.blast

## Simulate

In [1]:
import csv

import biom
from numpy.random import choice
import skbio.io

In [2]:
stool_sv = biom.load_table('stool_sv.biom')

In [3]:
stool_sv_map = {}
with open('stool_sv_map.blast') as blast_results:
    blast_reader = csv.reader(blast_results, csv.excel_tab)
    for row in blast_reader:
        assert row[0].endswith('blast_rocks')
        sv = row[0][:-len('blast_rocks')]
        if sv in stool_sv_map:
            assert stool_sv_map[sv] == row[1],\
                ' '.join([sv, stool_sv_map[sv], row[1]])
            continue
        stool_sv_map[sv] = row[1]

In [4]:
ref_seqs = {}
with open('99_otus_v4.fasta') as ref_fh:
    fasta_reader = skbio.io.read(ref_fh, 'fasta')
    for seq in fasta_reader:
        ref_seqs[seq.metadata['id']] = str(seq)

In [5]:
random_sample = stool_sv.ids()[choice(stool_sv.length())]
random_sample = stool_sv.filter([random_sample], inplace=False)
random_sample.filter(lambda v, _, __: v[0] > 1e-9, axis='observation', inplace=True)

156 x 1 <class 'biom.table.Table'> with 156 nonzero entries (100% dense)

In [6]:
with open('abundance.fasta', 'w') as a_fh:
    for row in random_sample.iter(axis='observation'):
        abundance, sv, _ = row
        abundance = int(abundance[0])
        if sv in stool_sv_map:
            a_fh.write('>' + sv + ';size=' + str(abundance) + '\n')
            a_fh.write(ref_seqs[stool_sv_map[sv]] + '\n')

In [7]:
%%bash
vsearch --rereplicate abundance.fasta --output prior_art.fasta
export PATH=$PATH:art_bin_MountRainier
art_illumina -ss HS25 -amp -i prior_art.fasta -l 150 -o post_art -c 1 -na
cat > post_art.manifest << END
sample-id,absolute-filepath,direction
sample-1,$PWD/post_art.fq,forward
END
qiime tools import --input-path post_art.manifest --output-path post_art.qza --type SampleData[SequencesWithQuality] --source-format SingleEndFastqManifestPhred33
mkdir tmp
qiime tools extract --output-dir tmp post_art.qza
mkdir post_art
mv tmp/*/data/* post_art
rm -r tmp/*
#qiime dada2 denoise-single --i-demultiplexed-seqs post_art.qza --o-representative-sequences dada2-seqs.qza --o-table simulated.qza --p-trunc-len 0 --p-n-threads 4 --p-no-hashed-feature-ids
#qiime deblur denoise-16S --i-demultiplexed-seqs post_art.qza --o-representative-sequences dada2-seqs.qza --o-table simulated.qza --p-trim-length -1 --p-sample-stats --o-stats stats.qza --p-jobs-to-start 4 --p-no-hashed-feature-ids


    ====================ART====================
             ART_Illumina (2008-2016)          
          Q Version 2.5.8 (June 6, 2016)       
     Contact: Weichun Huang <whduke@gmail.com> 
    -------------------------------------------

              Amplicon 5'-end sequencing simulation

Total CPU time used: 0.651425

The random seed for the run: 1521688503

Parameters used during run
	Read Length:	150
	Genome masking 'N' cutoff frequency: 	1 in 150
	# Reads per Amplion:       0
	Profile Type:             Combined
	ID Tag:                   

Quality Profile(s)
	First Read:   HiSeq 2500 Length 150 R1 (built-in profile) 

Output files

  FASTQ Sequence File:
	post_art.fq

Extracted to tmp/49e52d61-fe28-43c7-84b8-dfb683cb284a


vsearch v2.7.0_macos_x86_64, 16.0GB RAM, 8 cores
https://github.com/torognes/vsearch

Rereplicating 100%
Rereplicated 15620 reads from 156 amplicons
mkdir: tmp: File exists
mkdir: post_art: File exists


## Nasty
do not run - contains R code

In [1]:
%load_ext rpy2.ipython

In [2]:
inp_dir = "post_art"
out_path = "post_dada2.tsv"
filtered_dir = "tmp"
truncLen = 150
trimLeft = 0
maxEE = 2.0
truncQ = 2
chimeraMethod = "none"
minParentFold = 1.0
nthreads = 4
nreads_learn = 1000000

In [3]:
%R suppressWarnings(library(Rcpp))

array(['Rcpp', 'tools', 'stats', 'graphics', 'grDevices', 'utils',
       'datasets', 'methods', 'base'], 
      dtype='<U9')

In [4]:
%%R -i inp_dir,out_path,filtered_dir,truncLen,trimLeft,maxEE,truncQ,chimeraMethod,minParentFold,nthreads,nreads_learn
#!/usr/bin/env Rscript

###################################################
# This R script takes an input directory of .fastq.gz files
# and outputs a tsv file of the dada2 processed sequence
# table. It is intended for use with the QIIME2 plugin
# for DADA2.
#
# Ex: Rscript run_dada_single.R input_dir output.tsv filtered_dir 200 0 2.0 2 pooled 1.0 0 1000000
####################################################

####################################################
#             DESCRIPTION OF ARGUMENTS             #
####################################################
# NOTE: All numeric arguments should be zero or positive.
# NOTE: All numeric arguments save maxEE are expected to be integers.
# NOTE: Currently the filterered_dir must already exist.
# NOTE: ALL ARGUMENTS ARE POSITIONAL!
#
### FILE SYSTEM ARGUMENTS ###
#
# 1) File path to directory with the .fastq.gz files to be processed.
#    Ex: path/to/dir/with/fastqgzs
#
# 2) File path to output tsv file. If already exists, will be overwritten.
#    Ex: path/to/output_file.tsv
#
# 3) File path to directory in which to write the filtered .fastq.gz files. These files are intermediate
#               for the full workflow. Currently they remain after the script finishes.
#               Directory must already exist.
#    Ex: path/to/dir/with/fastqgzs/filtered
#
### FILTERING ARGUMENTS ###
#
# 4) truncLen - The position at which to truncate reads. Reads shorter
#               than truncLen will be discarded.
#               Special values: 0 - no truncation or length filtering.
#    Ex: 150
#
# 5) trimLeft - The number of nucleotides to remove from the start of
#               each read. Should be less than truncLen for obvious reasons.
#    Ex: 0
#
# 6) maxEE - Reads with expected errors higher than maxEE are discarded.
#    Ex: 2.0
#
# 7) truncQ - Reads are truncated at the first instance of quality score truncQ.
#                If the read is then shorter than truncLen, it is discarded.
#    Ex: 2
#
### CHIMERA ARGUMENTS ###
#
# 8) chimeraMethod - The method used to remove chimeras. Valid options are:
#               none: No chimera removal is performed.
#               pooled: All reads are pooled prior to chimera detection.
#               consensus: Chimeras are detect in samples individually, and a consensus decision
#                           is made for each sequence variant.
#    Ex: consensus
#
# 9) minParentFold - The minimum abundance of potential "parents" of a sequence being
#               tested as chimeric, expressed as a fold-change versus the abundance of the sequence being
#               tested. Values should be greater than or equal to 1 (i.e. parents should be more
#               abundant than the sequence being tested).
#    Ex: 1.0
#
### SPEED ARGUMENTS ###
#
# 10) nthreads - The number of threads to use.
#                 Special values: 0 - detect available cores and use all.
#    Ex: 1
#
# 11) nreads_learn - The minimum number of reads to learn the error model from.
#                 Special values: 0 - Use all input reads.
#    Ex: 1000000
#

cat(R.version$version.string, "\n")

# Input these above
# args <- commandArgs(TRUE)
# inp_dir <- args[[1]]
# out_path <- args[[2]]
# filtered_dir <- args[[3]]
# truncLen <- as.integer(args[[4]])
# trimLeft <- as.integer(args[[5]])
# maxEE <- as.numeric(args[[6]])
# truncQ <- as.integer(args[[7]])
# chimeraMethod <- args[[8]]
# minParentFold <- as.numeric(args[[9]])
# nthreads <- as.integer(args[[10]])
# nreads_learn <- as.integer(args[[11]])
errQuit <- function(mesg, status=1) {
  message("Error: ", mesg)
  q(status=status)
}

### VALIDATE ARGUMENTS ###

# Input directory is expected to contain .fastq.gz file(s)
# that have not yet been filtered and globally trimmed
# to the same length.
if(!dir.exists(inp_dir)) {
  errQuit("Input directory does not exist.")
} else {
  unfilts <- list.files(inp_dir, pattern=".fastq.gz$", full.names=TRUE)
  if(length(unfilts) == 0) {
    errQuit("No input files with the expected filename format found.")
  }
}

# Output path is to be a filename (not a directory) and is to be
# removed and replaced if already present.
if(dir.exists(out_path)) {
  errQuit("Output filename is a directory.")
} else if(file.exists(out_path)) {
  invisible(file.remove(out_path))
}

# Convert nthreads to the logical/numeric expected by dada2
if(nthreads < 0) {
  errQuit("nthreads must be non-negative.")
} else if(nthreads == 0) {
  multithread <- TRUE # detect and use all
} else if(nthreads == 1) {
  multithread <- FALSE
} else {
  multithread <- nthreads
}

### LOAD LIBRARIES ###
suppressWarnings(library(methods))
suppressWarnings(library(dada2))
cat("DADA2 R package version:", as.character(packageVersion("dada2")), "\n")

### TRIM AND FILTER ###
cat("1) Filtering ")
filts <- file.path(filtered_dir, basename(unfilts))
out <- suppressWarnings(filterAndTrim(unfilts, filts, truncLen=truncLen, trimLeft=trimLeft,
                                      maxEE=maxEE, truncQ=truncQ, rm.phix=TRUE, 
                                      multithread=multithread))
cat(ifelse(file.exists(filts), ".", "x"), sep="")
filts <- list.files(filtered_dir, pattern=".fastq.gz$", full.names=TRUE)
cat("\n")
if(length(filts) == 0) { # All reads were filtered out
  errQuit("No reads passed the filter (was truncLen longer than the read length?)", status=2)
}

### LEARN ERROR RATES ###
# Dereplicate enough samples to get nreads_learn total reads
cat("2) Learning Error Rates\n")
NREADS <- 0
drps <- vector("list", length(filts))
for(i in seq_along(filts)) {
  drps[[i]] <- derepFastq(filts[[i]])
  NREADS <- NREADS + sum(drps[[i]]$uniques)
  if(NREADS > nreads_learn) { break }
}
# Run dada in self-consist mode on those samples
dds <- vector("list", length(filts))
if(i==1) { # breaks list assignment
  dds[[1]] <- dada(drps[[1]], err=NULL, selfConsist=TRUE, multithread=multithread, VECTORIZED_ALIGNMENT=FALSE, SSE=1)
} else { # more than one sample, no problem with list assignment
  dds[1:i] <- dada(drps[1:i], err=NULL, selfConsist=TRUE, multithread=multithread, VECTORIZED_ALIGNMENT=FALSE, SSE=1)
}
err <- dds[[1]]$err_out
rm(drps)
cat("\n")

### PROCESS ALL SAMPLES ###
# Loop over rest in streaming fashion with learned error rates
cat("3) Denoise remaining samples ")
if(i < length(filts)) {
  for(j in seq(i+1,length(filts))) {
    drp <- derepFastq(filts[[j]])
    { sink("/dev/null"); dds[[j]] <- dada(drp, err=err, multithread=multithread, VECTORIZED_ALIGNMENT=FALSE, SSE=1); sink(); }
    cat(".")
  }
}
cat("\n")

# Make sequence table
seqtab <- makeSequenceTable(dds)

### Remove chimeras
cat("4) Remove chimeras (method = ", chimeraMethod, ")\n", sep="")
if(chimeraMethod %in% c("pooled", "consensus")) {
  seqtab.nochim <- removeBimeraDenovo(seqtab, method=chimeraMethod, minFoldParentOverAbundance=minParentFold, multithread=multithread)
} else { # No chimera removal, copy seqtab to seqtab.nochim
  seqtab.nochim <- seqtab
}

### REPORT READ FRACTIONS THROUGH PIPELINE ###
cat("5) Report read numbers through the pipeline\n")
# Handle edge cases: Samples lost in filtering; One sample
track <- cbind(out, matrix(0, nrow=nrow(out), ncol=2))
colnames(track) <- c("input", "filtered", "denoised", "non-chimeric")
passed.filtering <- track[,"filtered"] > 0
track[passed.filtering,"denoised"] <- rowSums(seqtab)
track[passed.filtering,"non-chimeric"] <- rowSums(seqtab.nochim)
head(track)
#write.table(track, out.track, sep="\t",
#            row.names=TRUE, col.names=col.names, quote=FALSE)

### WRITE OUTPUT AND QUIT ###
# Formatting as tsv plain-text sequence table table
cat("6) Write output\n")
seqtab.nochim <- t(seqtab.nochim) # QIIME has OTUs as rows
col.names <- basename(filts)
col.names[[1]] <- paste0("#OTU ID\t", col.names[[1]])
write.table(seqtab.nochim, out_path, sep="\t",
            row.names=TRUE, col.names=col.names, quote=FALSE)
saveRDS(seqtab.nochim, gsub("tsv", "rds", out_path)) ### TESTING

# Clearly not required in a jupyter cell
# q(status=0)

R version 3.4.1 (2017-06-30) 
DADA2 R package version: 1.6.0 
1) Filtering .
2) Learning Error Rates
Initializing error rates to maximum possible estimate.
Sample 1 - 15620 reads in 1745 unique sequences.
   selfConsist step 2 
   selfConsist step 3 
Convergence after  3  rounds.

3) Denoise remaining samples 
4) Remove chimeras (method = none)
5) Report read numbers through the pipeline
6) Write output


In [5]:
%%R
cat(dds[[1]]$map)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 1 45 1 1 1 43 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 28 26 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 29 41 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 30 16 1 1 1 31 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 42 7 8 37 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 39 32 33 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 47 38 34 1 35 31 20 1 1 2 2 2 2 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 44 51 13 13 5 40 46 49 36 52 2 1 2 2 4 9 6 6 9 4 4 4 2 2 2 2 2 2 2 2 6 2 2 2 2 2 3 3 3 3 3 3 3 3 4 3 2 1 1 1